In [ ]:
import json
import numpy as np
import pandas as pd

# Loading Data

In [ ]:
dataframes_path = "/data/processed/data_frames"
covid_data_path = "/data/raw/covid_data"
socio_economic_params_path = "/data/raw/socio_economic_params"

In [ ]:
with open("/src/main_configs.json", 'r') as f:
    main_configs = json.load(f)

## State Based Vaccination

In [ ]:
state_base_vaccination = pd.read_csv(f"{covid_data_path}/us_state_vaccinations.csv")

state_base_vaccination = state_base_vaccination[['date', 'location', 'people_vaccinated_per_hundred']]
state_base_vaccination['date'] = pd.to_datetime(state_base_vaccination['date'])
state_base_vaccination = state_base_vaccination[state_base_vaccination['date'] < '2022-01-01'].dropna()

In [ ]:
state_base_vaccination['location'] = state_base_vaccination['location'].replace({'New York State':'New York'})

In [ ]:
state_base_vaccination = state_base_vaccination[state_base_vaccination['location'].isin(main_configs['us_states_and_abbreviations'].keys())].sort_values('date')
state_base_vaccination['location'] = [main_configs['us_states_and_abbreviations'][i] for i in state_base_vaccination['location']]

In [ ]:
filtered_states = []
unique_states_in_vaccination = state_base_vaccination['location'].unique()
for i in main_configs['us_51_state']:
    if i in unique_states_in_vaccination:
        filtered_states.append(state_base_vaccination[state_base_vaccination['location'] == i].values[-1])
    
state_base_vaccination = pd.DataFrame(np.array(filtered_states), columns=state_base_vaccination.columns).drop(columns=['date'])

In [ ]:
state_base_vaccination = state_base_vaccination.rename(columns={'location':'state'})
state_base_vaccination = state_base_vaccination.join(fisher_exact_test_results_state.set_index('state'), on='state').drop(columns='padj')

In [ ]:
file_name = f"{dataframes_path}/state_based_vaccination_percentage_and_odd_ratios.parquet"
state_base_vaccination.to_parquet(file_name, index=False)